In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [62]:
#import libraries
import pandas as pd
import numpy as np
from sklearn import feature_extraction,model_selection,linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
import nltk
nltk.download('stopwords')
from nltk import word_tokenize

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
train_path='/kaggle/input/nlp-getting-started/train.csv'
test_path='/kaggle/input/nlp-getting-started/test.csv'
df=pd.read_csv(train_path,index_col='id')
df_t=pd.read_csv(test_path,index_col='id')
df.head()
df_t.isnull().sum()

keyword       26
location    1105
text           0
dtype: int64

In [64]:
#print first row of target rows with no disaster
df[df['target']==0]['text'].values[0]

"What's up man?"

In [65]:
#print first row of target rows with disaster
df[df['target']==1]['text'].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [66]:
#sample train vector creation
count_vectorizer=feature_extraction.text.CountVectorizer()
sample_train_vector=count_vectorizer.fit_transform(df['text'][0:5])

In [67]:
#for loop to print sample train vectors for first 5 tweets
for i in range(5):
    print(sample_train_vector[i].todense().shape)
    print(sample_train_vector[i].todense())


(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]
(1, 54)
[[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]]
(1, 54)
[[0 0 0 1 0 2 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 2 0 0 0 0 0 1 1 0 1 1 1 1 0 0
  0 2 0 0 0 1 0 0 0 0 0 2 0 0 0 1 0 0]]
(1, 54)
[[1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1]]
(1, 54)
[[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1]]


The CountVectorizer found 54 unique words in the first tweets. The code above compares the words in tweets with the list of 54 unique words and prints a non-zero number if the tweet contains one of the unique word list.

In [68]:
#create train and test vectors 
train_vectors=count_vectorizer.fit_transform(df['text'])
print(train_vectors.shape)
test_vectors=count_vectorizer.transform(df_t['text'])


(7613, 21637)


In [69]:
#model options
clf=linear_model.RidgeClassifier()
clf1=LogisticRegression(max_iter=750)
clf2=RandomForestClassifier(n_estimators=500)
clf3=MLPClassifier(hidden_layer_sizes=(200,100,50,25),max_iter=200)
clf6=KNeighborsClassifier(n_neighbors=7)

In [70]:
params={'max_iter':(2000,2500,3000),'solver':('lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'),'fit_intercept':(True,False),'penalty':('l2','l1','elasticnet')}
clf4=GridSearchCV(estimator=LogisticRegression(max_iter=750),param_grid=params,cv=3)
clf4.fit(train_vectors,df['target'])

GridSearchCV(cv=3, estimator=LogisticRegression(max_iter=750),
             param_grid={'fit_intercept': (True, False),
                         'max_iter': (2000, 2500, 3000),
                         'penalty': ('l2', 'l1', 'elasticnet'),
                         'solver': ('lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga')})

In [71]:
#print best_params_ and best_score_ attributes from the GridSearchCV
print(clf4.best_params_)
print(clf4.best_score_)

{'fit_intercept': True, 'max_iter': 2000, 'penalty': 'l2', 'solver': 'sag'}
0.7258666819073509


In [72]:
#apply the best params to logistic regression
clf5=LogisticRegression(max_iter=2000,fit_intercept=True,penalty='l2',solver='lbfgs')


In [73]:
scores=model_selection.cross_val_score(clf3,train_vectors,df['target'],cv=3,scoring='f1',n_jobs=6)
scores

array([0.62347784, 0.60556316, 0.65059222])

In [74]:
#fitting the model chosen based cv score
clf5.fit(train_vectors,df['target'])

LogisticRegression(max_iter=2000)

In [75]:
#run predictions
pred=clf5.predict(test_vectors)
pred=pd.DataFrame(pred,index=df_t.index)
pred.columns=['target']
pred

,target
id,
0,1
2,1
3,1
9,0
11,1
...,...
10861,1
10865,1
10868,1


In [76]:
#create submission file
pred.to_csv('submission.csv',sep=',')